In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
from cc_utils import _get_clip_labels

# plot
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
pio.templates.default = 'plotly_white'
from plot_utils import _hex_to_rgb, _plot_ts

#colors = px.colors.qualitative.Plotly
colors = ["#D55E00", "#009E73", "#56B4E9", "#E69F00"]

In [ ]:
class ARGS():
    roi = 300
    net = 7
    subnet = 'wb'
    zscore = 1
    k_fold = 10
    k_hidden = 150
    batch_size = 16
    num_epochs = 50
    train_size = 100
    
    #lstm
    lstm_layers = 1
    
    #tcn
    tcn_wind = 10
    
    #ff
    ff_layers = 5

In [ ]:
args = ARGS()

In [ ]:
RES_DIR = {}
RES_DIR['ff'] = 'results/clip_ff'
RES_DIR['tcn'] = 'results/clip_tcn'
RES_DIR['lstm'] = 'results/clip_lstm'

In [ ]:
def _get_results(args, model):
    
    if model=='lstm':
        k_layers = args.lstm_layers
    elif model=='ff':
        k_layers = args.ff_layers
    
    if model=='tcn':
        load_path = (RES_DIR['tcn'] + 
            '/roi_%d_net_%d' %(args.roi, args.net) + 
            '_nw_%s' %(args.subnet) +
            '_trainsize_%d' %(args.train_size) +
            '_kfold_%d_k_hidden_%d' %(args.k_fold, args.k_hidden) +
            '_k_wind_%d_batch_size_%d' %(args.tcn_wind, args.batch_size) +
            '_num_epochs_%d_z_%d.pkl' %(args.num_epochs, args.zscore))
        
    else:
        load_path = (RES_DIR[model] + 
            '/roi_%d_net_%d' %(args.roi, args.net) + 
            '_nw_%s' %(args.subnet) +
            '_trainsize_%d' %(args.train_size) +
            '_kfold_%d_k_hidden_%d' %(args.k_fold, args.k_hidden) +
            '_k_layers_%d_batch_size_%d' %(k_layers, args.batch_size) +
            '_num_epochs_%d_z_%d.pkl' %(args.num_epochs, args.zscore))
    
    with open(load_path, 'rb') as f:
        r = pickle.load(f)
    
    return r

In [ ]:
def _compare_mean(models, mode='train_mode'):
    '''
    compare mean accuracy for each model
    '''
    fig = go.Figure()
    ticktext = []
    for ii, m in enumerate(models):
        if m[0]=='ff':
            args.ff_layers = m[1]
        elif m[0]=='tcn':
            args.tcn_wind = m[1]
        elif m[0]=='lstm':
            args.lstm_layers = m[1]
        ticktext.append(m[2])
        
        r = _get_results(args, m[0])[mode]
        if mode=='train_mode':
            tag = 'val'
            multiplier = 3
        elif mode=='test_mode':
            tag = 'test'
            multiplier = 1
            
        y = np.mean(r[tag])
        err_y = 1/np.sqrt(args.k_fold)*np.std(r[tag])

        bar = go.Bar(x=[ii], y=[y],
            error_y=dict(type='data',
                array=[multiplier*err_y], width=9),
            name=m[2],
            showlegend=True,
            marker_color=colors[ii])
        fig.add_trace(bar)

    fig.update_yaxes(range=[0, 1],
        title=dict(text='Clip Prediction Accuracy',
            font_size=25),
        gridwidth=1, gridcolor='#bfbfbf',
        tickfont=dict(size=20))
    fig.update_xaxes(showticklabels=False)
    fig.update_layout(height=500, width=450,
        font_color='black',
        legend_orientation='h',
        legend_font_size=20,
        legend_x=-0.1)
    
    return fig

In [ ]:
def _compare_temporal(models, clipnames, mode='train_mode'):
    '''
    compare temporal accuracy for each model
    '''
    k_class = len(clip_names)
    k_rows = int(np.ceil(k_class/3))
    k_cols = 3
    fig = make_subplots(rows=k_rows, cols=k_cols, 
        subplot_titles=clip_names, print_grid=False)
    
    fig_clip = {}
    for clip in clipnames:
        fig_clip[clip] = go.Figure()
        
    for ii, m in enumerate(models):
        if m[0]=='ff':
            args.ff_layers = m[1]
        elif m[0]=='tcn':
            args.tcn_wind = m[1]
        elif m[0]=='lstm':
            args.lstm_layers = m[1]
        
        r = _get_results(args, m[0])[mode]
        if mode=='train_mode':
            tag = 't_val'
        elif mode=='test_mode':
            tag = 't_test'
        name = m[2]

        max_time = -100
        for jj in range(k_class):
            row = int(jj/k_cols) + 1
            col = (jj%k_cols) + 1

            showlegend = False
            if jj == 0:
                showlegend = True

            acc = r[tag][jj]
            ts = {'mean': np.mean(acc, axis=0),
                'ste': 1/np.sqrt(len(acc)) * np.std(acc, axis=0)}

            plotter = _plot_ts(ts, colors[ii],
                showlegend=showlegend, name=name)
            for trace in plotter:
                fig.add_trace(plotter[trace], row, col)
            for trace in plotter:
                fig_clip[clipnames[jj]].add_trace(plotter[trace])

            if len(ts['mean']) > max_time:
                max_time = len(ts['mean'])

    fig.update_layout(height=int(250*k_rows), width=750,
        legend_orientation='h')
    fig.update_xaxes(range=[0, max_time], dtick=30, 
        title_text='time (in s)',
        showgrid=False,
        autorange=False)
    fig.update_yaxes(range=[0, 1], dtick=0.2,
        gridwidth=1, gridcolor='#bfbfbf',
        autorange=False)
    
    return fig, fig_clip

In [ ]:
clip_y = _get_clip_labels()
k_class = len(np.unique(list(clip_y.values())))
print('number of classes = %d' %k_class)

clip_names = np.zeros(k_class).astype(str)
clip_names[0] = 'testretest'
for key, item in clip_y.items():
    if item!=0:
        clip_names[item] = key

# TEST MODE

In [ ]:
table = []
for model, file in RES_DIR.items():
    entry = {}
    entry['model'] = model
    r = _get_results(args, model)['test_mode']
    entry['train'] = np.mean(r['train'])
    entry['test'] = np.mean(r['test'])
    
    table.append(entry)

df = pd.DataFrame(table)
df

In [ ]:
# model: (name, param, abbrev)

models = [('ff', 5, 'FF: (D=5)'), 
    ('tcn', 10, 'TCN: (W=10)'), 
    ('tcn', 50, 'TCN: (W=50)'),
    ('lstm', 1, 'LSTM')]

fig = _compare_mean(models, mode='test_mode')
fig.show()
fig_mean = go.Figure(fig)

In [ ]:
# model: (name, param, abbrev)

models = [('ff', 5, 'FF: (D=5)'), 
    ('tcn', 10, 'TCN: (W=10)'), 
    ('tcn', 50, 'TCN: (W=50)'),
    ('lstm', 1, 'LSTM')]

fig, fig_clip = _compare_temporal(models, clip_names,
    mode='test_mode')
fig.show()

In [ ]:
clip = 'socialnet'
fig = fig_clip[clip]
fig.update_yaxes(range=[0, 1],
#     title=dict(text='Clip Prediction Accuracy',
#         font_size=25),
    gridwidth=1, gridcolor='#bfbfbf',
    tickfont=dict(size=20))
fig.update_xaxes(tickfont=dict(size=20),
    dtick=30,
    title=dict(text='Time (in s)', font_size=20))
fig.update_layout(height=470, width=550,
    showlegend=False,
    font_color='black',
    title='')
fig.show()